In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-03-06 01:35:36,994 INFO: Initializing external client
2025-03-06 01:35:36,995 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 01:35:38,026 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691
Fetching data from 2025-02-05 06:35:36.992742+00:00 to 2025-03-06 05:35:36.992742+00:00
2025-03-06 01:36:35,116 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 209, in __init__
    self._health_check()
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retrying.py", line 266, in call
    raise attempt.get()
          ^^^^^^^^^^^^^
  File "c:\

2025-03-06 01:36:41,820 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 394, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 459, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retry

AttributeError: 'ArrowFlightClient' object has no attribute '_server_version'

In [ ]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 07:00:00
1,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,3,2025-03-05 07:00:00
2,4,6,2,3,2,0,1,3,2,1,...,2,5,3,3,0,0,0,1,4,2025-03-05 07:00:00
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,6,2025-03-05 07:00:00
4,6,4,2,1,4,1,2,1,2,2,...,1,1,0,2,0,2,0,2,7,2025-03-05 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-05 07:00:00
248,1,4,1,0,3,2,5,0,1,2,...,1,1,0,0,0,1,2,0,260,2025-03-05 07:00:00
249,8,15,14,12,23,23,25,28,23,29,...,13,4,1,0,0,0,2,0,261,2025-03-05 07:00:00
250,151,147,158,114,84,84,65,73,99,89,...,5,1,2,2,4,7,18,79,262,2025-03-05 07:00:00


In [ ]:
predictions = fetch_next_hour_predictions()

2025-03-06 01:33:18,595 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 01:33:18,609 INFO: Initializing external client
2025-03-06 01:33:18,610 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 01:33:19,290 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691
2025-03-06 01:34:15,549 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 209, in __init__
    self._health_check()
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retrying.py", line 266, in call
    raise attempt.get()
          ^^^^^^^^^^

2025-03-06 01:34:20,612 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 394, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 459, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\retry

AttributeError: 'ArrowFlightClient' object has no attribute '_server_version'

In [ ]:
predictions


,pickup_location_id,predicted_demand,pickup_hour
3263,71,1.0,2025-03-05 07:00:00+00:00
3264,130,1.0,2025-03-05 07:00:00+00:00
3265,43,92.0,2025-03-05 07:00:00+00:00
3266,14,1.0,2025-03-05 07:00:00+00:00
3267,42,3.0,2025-03-05 07:00:00+00:00
...,...,...,...
3509,163,62.0,2025-03-05 07:00:00+00:00
3510,213,1.0,2025-03-05 07:00:00+00:00
3511,261,23.0,2025-03-05 07:00:00+00:00
3512,48,93.0,2025-03-05 07:00:00+00:00


In [ ]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [ ]:
top10

[132, 161, 230, 170, 142, 234, 249, 164, 186, 236]

In [ ]:
top10

[132, 161, 230, 170, 142, 234, 249, 164, 186, 236]

In [ ]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
246    False
247    False
248    False
249    False
250    False
Name: pickup_location_id, Length: 251, dtype: bool

In [ ]:
features_cp = features.copy()

In [ ]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

In [ ]:
historical_df

,datetime,rides
0,2025-02-04 07:00:00,85.0
1,2025-02-04 08:00:00,166.0
2,2025-02-04 09:00:00,256.0
3,2025-02-04 10:00:00,208.0
4,2025-02-04 11:00:00,257.0
...,...,...
668,2025-03-04 03:00:00,3.0
669,2025-03-04 04:00:00,3.0
670,2025-03-04 05:00:00,10.0
671,2025-03-04 06:00:00,65.0
